In [ ]:
import nanohubremote as nr
import pandas as pd
import pandas as pd
import plotly.graph_objs as go
import os
import numpy as np

In [ ]:
auth_data = {
    'grant_type' : 'tool',
}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"] 

In [ ]:
import json
session = nr.Session(auth_data)
search = {
    'tool':'cellrelaxdft', 
    'filters':json.dumps([
        {'field':'input.mp_id','operation':'=','value':'mp-1009127'},
        {'field':'output.energy','operation':'>','value':-2492.0},
        {'field':'output.pressure','operation':'>','value':500},
        {'field':'output.pressure','operation':'<','value':700},
    ]),
    'results':json.dumps([
        'input.k_x',
        'input.k_y',
        'input.k_z',
        'output.pressure',
        'output.energy',
        'output.lattice_vectors',
        'output.lattice_parameters',
    ]),
    'simtool' : True,
    'revision' : 49,
}
req_json = session.requestPost('results/dbexplorer/search', data=search)
req_json = req_json.json()
data = pd.DataFrame([p for p in req_json["results"]])


In [ ]:
data

In [ ]:
pd.options.plotting.backend = "plotly"
data.plot.scatter(x="output.pressure", y="output.energy")

In [ ]:
search = {
    'tool':'pntoy', 
    'filters':json.dumps([        
        {'field':'input.p_len','value':4,'operation':'=='},
        {'field':'input.n_len','value':4,'operation':'=='},
        {'field':'input.temperature','value':300,'operation':'=='},
        {'field':'input.i_len','value':0,'operation':'=='},
        {'field':'input.materialp','value':"Si",'operation':'=='},
        {'field':'input.impurity','value':'false','operation':'=='},
    ]),
    'results':json.dumps([
        'input.Na', 
        'input.Nd',
        'output.p',
    ]),    
    'limit':10,    
}
req_json = session.requestPost('results/dbexplorer/search', data=search)
req_json = req_json.json()
df = pd.DataFrame(req_json['results'])


In [ ]:
data = df.apply(lambda a : {"x":a['output.p']['xaxis'],"y":a['output.p']['yaxis'], 'name':a['squid'],'type':'scatter'}, axis=1).tolist()

In [ ]:
go.FigureWidget(data = data, layout={'yaxis':{'type':'log', 'exponentformat':"e"}})